In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import requests
import time
import tqdm
from datetime import datetime
from google.cloud.logging import DESCENDING, Client
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['font.family'] = 'monospace'

read delivered payloads for the last 24 hours from the db.

```
psql -p 5433 "host=0.0.0.0 sslmode=disable dbname=defaultdb user=readonly" -t -c "select slot,block_hash,proposer_pubkey,inserted_at from mainnet_payload_delivered where slot>=6287355;" -o cancel/delivered-apr24.txt
```

slot range $[6287356,6294551]$.

# delivered payloads

In [2]:
# read data from file.
dlvd = pd.read_csv('delivered-apr24.txt', delimiter='|', names=['slot','block_hash','proposer_pubkey','inserted_at'])
dlvd['inserted_at'] = pd.to_datetime(dlvd['inserted_at'], format="%Y-%m-%d%H:%M:%S.%f")
dlvd['slot_boundary'] = pd.to_datetime(1606824023 + dlvd['slot'] * 12, unit='s')
dlvd.head()

,slot,block_hash,proposer_pubkey,inserted_at,slot_boundary
0,6287356,0x0f58d3a1b835ae053388804701180f246611b8939572...,0x8656dab6bae6f46037862a8954af4f72d97b63866f65...,2023-04-23 17:51:37.291059,2023-04-23 17:51:35
1,6287363,0xd8c3792a70a517ee58df86a78d49a28654510038afe9...,0x8c9dd8f50a31f51a0525d1a53697716c838c45060bb3...,2023-04-23 17:53:01.124746,2023-04-23 17:52:59
2,6287371,0xa398751b66f4abe3fc09c244faa35c7d5160492b36b1...,0x9076ea0749534137aa85a41bedd3e55a008d8e8a00eb...,2023-04-23 17:54:37.072391,2023-04-23 17:54:35
3,6287375,0x17518a7788f5683c516e1bffa1bd9c5d2d833e527730...,0xb9845cceda7505b05ad2edfd474805d66bd1ecc4494e...,2023-04-23 17:55:26.290921,2023-04-23 17:55:23
4,6287376,0x91037fcbfd76866cca8c2e60c0a021c956545de16998...,0x8ab0403a2931163a618d066d38364969841d785f0503...,2023-04-23 17:55:37.285840,2023-04-23 17:55:35


# getPayload calls from GCP.

NOTE: we are getting just the first call to getPayload.

In [38]:
client = Client(project='ultra-sound-relay')

/usr/local/lib/python3.8/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [ ]:
getPayloads = pd.DataFrame(columns=['slot', 'ip', 'ua', 'reqTime'])

In [20]:
for v in dlvd.iterrows():
    if v[1]['slot'] < 6287906:
        time.sleep(0.5)
        # get successful getPayload calls.
        inAt = v[1]['inserted_at']
        print('querying logs for slot {} inserted at {}'.format(v[1]['slot'], v[1]['inserted_at']), end='\r')
        gpFilter = '"https://relay.ultrasound.money/eth/v1/builder/blinded_block" ' + \
                   'AND timestamp>=' + '\"' + (inAt - pd.Timedelta('12s')).strftime('%Y-%m-%dT%H:%M:%S.%fZ') + '\" ' + \
                   'AND timestamp<=' + '\"' + (inAt).strftime('%Y-%m-%dT%H:%M:%S.%fZ') + '\"'
        entries = False
        for e in client.list_entries(filter_=gpFilter):
            entries = True
            out = {}
            out['slot'] = v[1]['slot']
            out['ip'] = e.http_request['remoteIp']
            out['ua'] = e.http_request['userAgent']
            out['reqTime'] = e.timestamp 
            getPayloads = getPayloads.append(out, ignore_index=True)
        if not entries:
            print("\n *** no entries found for slot: {}".format(v[1]['slot']))
            print(gpFilter)

In [32]:
getPayloads = getPayloads.sort_values(['slot', 'reqTime'], ascending=[True, True])
getPayloads = getPayloads.drop_duplicates(['slot'])
getPayloads.reset_index(drop=True)
getPayloads.head()

,slot,ip,ua,reqTime
2782,6287356,34.77.3.1,Go-http-client/1.1,2023-04-23 17:51:35.004310+00:00
2783,6287363,89.149.218.12,mev-boost/v1.5.0 Lighthouse/v4.1.0-693886b,2023-04-23 17:52:59.508839+00:00
2784,6287371,57.128.32.19,mev-boost/v1.5.0,2023-04-23 17:54:35.524251+00:00
2785,6287375,158.140.135.239,mev-boost/v1.5.0 Prysm/v4.0.3/f4681fde1946a86c...,2023-04-23 17:55:23.986769+00:00
2786,6287376,3.74.185.146,Go-http-client/1.1,2023-04-23 17:55:35.012735+00:00


In [33]:
len(set(getPayloads.slot)), len(dlvd), len(getPayloads.slot)

(2658, 2658, 2658)

In [35]:
getPayloadsFull = pd.merge(getPayloads, dlvd, on='slot')
getPayloadsFull

,slot,ip,ua,reqTime,block_hash,proposer_pubkey,inserted_at,slot_boundary
0,6287356,34.77.3.1,Go-http-client/1.1,2023-04-23 17:51:35.004310+00:00,0x0f58d3a1b835ae053388804701180f246611b8939572...,0x8656dab6bae6f46037862a8954af4f72d97b63866f65...,2023-04-23 17:51:37.291059,2023-04-23 17:51:35
1,6287363,89.149.218.12,mev-boost/v1.5.0 Lighthouse/v4.1.0-693886b,2023-04-23 17:52:59.508839+00:00,0xd8c3792a70a517ee58df86a78d49a28654510038afe9...,0x8c9dd8f50a31f51a0525d1a53697716c838c45060bb3...,2023-04-23 17:53:01.124746,2023-04-23 17:52:59
2,6287371,57.128.32.19,mev-boost/v1.5.0,2023-04-23 17:54:35.524251+00:00,0xa398751b66f4abe3fc09c244faa35c7d5160492b36b1...,0x9076ea0749534137aa85a41bedd3e55a008d8e8a00eb...,2023-04-23 17:54:37.072391,2023-04-23 17:54:35
3,6287375,158.140.135.239,mev-boost/v1.5.0 Prysm/v4.0.3/f4681fde1946a86c...,2023-04-23 17:55:23.986769+00:00,0x17518a7788f5683c516e1bffa1bd9c5d2d833e527730...,0xb9845cceda7505b05ad2edfd474805d66bd1ecc4494e...,2023-04-23 17:55:26.290921,2023-04-23 17:55:23
4,6287376,3.74.185.146,Go-http-client/1.1,2023-04-23 17:55:35.012735+00:00,0x91037fcbfd76866cca8c2e60c0a021c956545de16998...,0x8ab0403a2931163a618d066d38364969841d785f0503...,2023-04-23 17:55:37.285840,2023-04-23 17:55:35
...,...,...,...,...,...,...,...,...
2653,6294530,167.99.164.32,mev-boost/v1.5.0,2023-04-24 17:46:24.047788+00:00,0x7041bca1f03039be5eb000e7cd55bb9a0e8fc2b9eb26...,0xa1fca56e908f2a3610cc070b6a57d373687b428b1a09...,2023-04-24 17:46:25.658167,2023-04-24 17:46:23
2654,6294531,35.88.134.71,mev-boost/v1.5.0 Go-http-client/1.1,2023-04-24 17:46:36.435164+00:00,0x18bcbf52e094cf1d598c2d5c560db6160774d0f03387...,0xa62a2c65c8e6a41b258c50cbff2d8f683af02b1145df...,2023-04-24 17:46:38.090339,2023-04-24 17:46:35
2655,6294549,34.82.140.105,mev-boost/v1.5.0 okhttp/4.10.0,2023-04-24 17:50:11.566013+00:00,0x1c6d90901205d0c0f7e1a35c375e313cd31713aaf927...,0xa2a207261959aba515f142e26b0f3262083ebf77716f...,2023-04-24 17:50:13.325541,2023-04-24 17:50:11
2656,6294550,20.203.252.23,mev-boost/v1.5.0 okhttp/4.10.0,2023-04-24 17:50:23.533425+00:00,0xb99ce21ca2f7ae186c887a65d940c8f9b77ee408871f...,0x85c7fc170e939c1e1be0deb43bc28cf787263a9c60d9...,2023-04-24 17:50:24.983845,2023-04-24 17:50:23


In [53]:
getPayloadsFull.to_csv('gpFull.csv', index=False)

# getHeader calls from GCP

In [41]:
getHeaders = pd.DataFrame(columns=['slot', 'ip', 'ua', 'reqTime'])

In [43]:
for v in getPayloadsFull.iterrows():
    time.sleep(1)
    # get successful getPayload calls.
    inAt = v[1]['inserted_at']
    print('querying logs for slot {} inserted at {}'.format(v[1]['slot'], v[1]['inserted_at']), end='\r')
    ghFilter = 'resource.type="http_load_balancer" ' + \
               'httpRequest.status=200 ' + \
               '"https://relay.ultrasound.money/eth/v1/builder/header/{}" '.format(v[1]['slot']) + \
               'jsonPayload.remoteIp="{}" '.format(v[1]['ip']) + \
               'AND timestamp>=' + '\"' + (inAt - pd.Timedelta('12s')).strftime('%Y-%m-%dT%H:%M:%S.%fZ') + '\" ' + \
               'AND timestamp<=' + '\"' + (inAt).strftime('%Y-%m-%dT%H:%M:%S.%fZ') + '\"'
    entries = False
    for e in client.list_entries(filter_=ghFilter):
        entries = True
        out = {}
        out['slot'] = v[1]['slot']
        out['ip'] = e.http_request['remoteIp']
        out['ua'] = e.http_request['userAgent']
        out['reqTime'] = e.timestamp 
        getHeaders = getHeaders.append(out, ignore_index=True)
    if not entries:
        print("*** no entries found for slot: {}".format(v[1]['slot']), end='\r')

In [67]:
ghFilter

'resource.type="http_load_balancer" httpRequest.status=200 "https://relay.ultrasound.money/eth/v1/builder/header/6294551" jsonPayload.remoteIp="34.249.7.41" AND timestamp>="2023-04-24T17:50:24.994254Z" AND timestamp<="2023-04-24T17:50:36.994254Z"'

In [45]:
len(getHeaders)

2125

In [46]:
getHeaders.to_csv('ghFull.txt', index=False)

In [56]:
getPayloadsFull.head()

,slot,ip,ua,reqTime,block_hash,proposer_pubkey,inserted_at,slot_boundary
0,6287356,34.77.3.1,Go-http-client/1.1,2023-04-23 17:51:35.004310+00:00,0x0f58d3a1b835ae053388804701180f246611b8939572...,0x8656dab6bae6f46037862a8954af4f72d97b63866f65...,2023-04-23 17:51:37.291059,2023-04-23 17:51:35
1,6287363,89.149.218.12,mev-boost/v1.5.0 Lighthouse/v4.1.0-693886b,2023-04-23 17:52:59.508839+00:00,0xd8c3792a70a517ee58df86a78d49a28654510038afe9...,0x8c9dd8f50a31f51a0525d1a53697716c838c45060bb3...,2023-04-23 17:53:01.124746,2023-04-23 17:52:59
2,6287371,57.128.32.19,mev-boost/v1.5.0,2023-04-23 17:54:35.524251+00:00,0xa398751b66f4abe3fc09c244faa35c7d5160492b36b1...,0x9076ea0749534137aa85a41bedd3e55a008d8e8a00eb...,2023-04-23 17:54:37.072391,2023-04-23 17:54:35
3,6287375,158.140.135.239,mev-boost/v1.5.0 Prysm/v4.0.3/f4681fde1946a86c...,2023-04-23 17:55:23.986769+00:00,0x17518a7788f5683c516e1bffa1bd9c5d2d833e527730...,0xb9845cceda7505b05ad2edfd474805d66bd1ecc4494e...,2023-04-23 17:55:26.290921,2023-04-23 17:55:23
4,6287376,3.74.185.146,Go-http-client/1.1,2023-04-23 17:55:35.012735+00:00,0x91037fcbfd76866cca8c2e60c0a021c956545de16998...,0x8ab0403a2931163a618d066d38364969841d785f0503...,2023-04-23 17:55:37.285840,2023-04-23 17:55:35


# join the two

In [62]:
merged = pd.merge(getHeaders, getPayloadsFull, on=['ip', 'slot'], how='left')
merged.head()

,slot,ip,ua_x,reqTime_x,ua_y,reqTime_y,block_hash,proposer_pubkey,inserted_at,slot_boundary
0,6287363,89.149.218.12,mev-boost/v1.5.0 Lighthouse/v4.1.0-693886b,2023-04-23 17:52:59.224636+00:00,mev-boost/v1.5.0 Lighthouse/v4.1.0-693886b,2023-04-23 17:52:59.508839+00:00,0xd8c3792a70a517ee58df86a78d49a28654510038afe9...,0x8c9dd8f50a31f51a0525d1a53697716c838c45060bb3...,2023-04-23 17:53:01.124746,2023-04-23 17:52:59
1,6287371,57.128.32.19,mev-boost/v1.5.0,2023-04-23 17:54:35.214206+00:00,mev-boost/v1.5.0,2023-04-23 17:54:35.524251+00:00,0xa398751b66f4abe3fc09c244faa35c7d5160492b36b1...,0x9076ea0749534137aa85a41bedd3e55a008d8e8a00eb...,2023-04-23 17:54:37.072391,2023-04-23 17:54:35
2,6287375,158.140.135.239,mev-boost/v1.5.0 Prysm/v4.0.3/f4681fde1946a86c...,2023-04-23 17:55:23.190772+00:00,mev-boost/v1.5.0 Prysm/v4.0.3/f4681fde1946a86c...,2023-04-23 17:55:23.986769+00:00,0x17518a7788f5683c516e1bffa1bd9c5d2d833e527730...,0xb9845cceda7505b05ad2edfd474805d66bd1ecc4494e...,2023-04-23 17:55:26.290921,2023-04-23 17:55:23
3,6287363,89.149.218.12,mev-boost/v1.5.0 Lighthouse/v4.1.0-693886b,2023-04-23 17:52:59.224636+00:00,mev-boost/v1.5.0 Lighthouse/v4.1.0-693886b,2023-04-23 17:52:59.508839+00:00,0xd8c3792a70a517ee58df86a78d49a28654510038afe9...,0x8c9dd8f50a31f51a0525d1a53697716c838c45060bb3...,2023-04-23 17:53:01.124746,2023-04-23 17:52:59
4,6287371,57.128.32.19,mev-boost/v1.5.0,2023-04-23 17:54:35.214206+00:00,mev-boost/v1.5.0,2023-04-23 17:54:35.524251+00:00,0xa398751b66f4abe3fc09c244faa35c7d5160492b36b1...,0x9076ea0749534137aa85a41bedd3e55a008d8e8a00eb...,2023-04-23 17:54:37.072391,2023-04-23 17:54:35


In [64]:
merged = merged.rename(columns={'reqTime_x': 'header_req', 
                                'reqTime_y': 'payload_req',
                                'ua_x': 'header_ua',  
                                'ua_y': 'payload_ua',
                       })
merged = merged[['slot', 'ip', 'header_ua', 'payload_ua', 'slot_boundary', 'header_req', 'payload_req', 'block_hash', 'proposer_pubkey']]

for col in ['slot_boundary', 'header_req', 'payload_req']:
    merged[col] = pd.to_datetime(merged[col]).dt.strftime("%Y-%m-%d %H:%M:%S.%f")
    merged[col] = pd.to_datetime(merged[col])
merged = merged.drop_duplicates(['header_req'])
merged = merged.sort_values(['slot', 'header_req'], ascending=[True, True])
merged.reset_index(drop=True)
merged.head()

,slot,ip,header_ua,payload_ua,slot_boundary,header_req,payload_req,block_hash,proposer_pubkey
0,6287363,89.149.218.12,mev-boost/v1.5.0 Lighthouse/v4.1.0-693886b,mev-boost/v1.5.0 Lighthouse/v4.1.0-693886b,2023-04-23 17:52:59,2023-04-23 17:52:59.224636,2023-04-23 17:52:59.508839,0xd8c3792a70a517ee58df86a78d49a28654510038afe9...,0x8c9dd8f50a31f51a0525d1a53697716c838c45060bb3...
1,6287371,57.128.32.19,mev-boost/v1.5.0,mev-boost/v1.5.0,2023-04-23 17:54:35,2023-04-23 17:54:35.214206,2023-04-23 17:54:35.524251,0xa398751b66f4abe3fc09c244faa35c7d5160492b36b1...,0x9076ea0749534137aa85a41bedd3e55a008d8e8a00eb...
2,6287375,158.140.135.239,mev-boost/v1.5.0 Prysm/v4.0.3/f4681fde1946a86c...,mev-boost/v1.5.0 Prysm/v4.0.3/f4681fde1946a86c...,2023-04-23 17:55:23,2023-04-23 17:55:23.190772,2023-04-23 17:55:23.986769,0x17518a7788f5683c516e1bffa1bd9c5d2d833e527730...,0xb9845cceda7505b05ad2edfd474805d66bd1ecc4494e...
6,6287378,20.203.252.23,mev-boost/v1.5.0 okhttp/4.10.0,mev-boost/v1.5.0 okhttp/4.10.0,2023-04-23 17:55:59,2023-04-23 17:55:59.122244,2023-04-23 17:55:59.540486,0xb34b74c7fdf4a74110ad4d35ab05da18855645522258...,0x95d75b5910790cb42f7947ab14c679267e8f73be1a77...
7,6287380,34.249.7.41,mev-boost/v1.5.0 Prysm/v4.0.2/f4681fde1946a86c...,mev-boost/v1.5.0 Prysm/v4.0.2/f4681fde1946a86c...,2023-04-23 17:56:23,2023-04-23 17:56:23.347438,2023-04-23 17:56:23.565121,0x5b0dbb355fcdce011e2624312aa4420fe951a69f2509...,0x971860fbd43fff1cb91ac0a35bc4c7efd6de0b12331e...


In [66]:
len(merged), len(set(merged.slot))

(2122, 2121)

In [68]:
merged.to_csv('merged.txt', index=False)

# figures: 